In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Label

Labels of enhancers remain strongly unbalance, even with dropping. Almost all values of enhancers labels are 0 (third quartile is 0).

I decided to set promoters threshold equal to 1 and don't drop values between 0 and 1 because this improve balance between classes but not enough to eliminate samples from the dataset. 
About enhancers the threshold is set to 0 because this leads classes to be as balanced as possible.

I'm not focusing on the biological meaning of my choices, the threshold are chosen to obtain a dataset useful for learning. 

In [ ]:
promoters_labels = pd.read_csv('./pre_processed_dataset/labels_promoters.csv', index_col = [0,1,2,3])
enhancers_labels = pd.read_csv('./pre_processed_dataset/labels_enhancers.csv', index_col = [0,1,2,3])
promoters = pd.read_csv('./pre_processed_dataset/epigenomes_promoters.csv', index_col = [0,1,2,3])
enhancers = pd.read_csv('./pre_processed_dataset/epigenomes_enhancers.csv', index_col = [0,1,2,3])

epigenomes = {
    "promoters": promoters,
    "enhancers": enhancers
}

labels = {
    "promoters": promoters_labels,
    "enhancers": enhancers_labels
}

### Features distribution

In [11]:
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
def get_top_most_different(dist, n:int):
    return np.argsort(-np.mean(dist, axis=1).flatten())[:n]

In [3]:
top_number = 5

for region, x in epigenomes.items():
    dist = euclidean_distances(x.T)
    most_distance_columns_indices = get_top_most_different(dist, top_number)
    columns = x.columns[most_distance_columns_indices]
    fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(25, 5))
    print(f"Top {top_number} different features from {region}.")
    for column, axis in zip(columns, axes.flatten()):
        head, tail = x[column].quantile([0.05, 0.95]).values.ravel()
        
        mask = ((x[column] < tail) & (x[column] > head)).values
        
        cleared_x = x[column][mask]
        cleared_y = labels[region].values.ravel()[mask]
        
        cleared_x[cleared_y==0].hist(ax=axis, bins=20)
        cleared_x[cleared_y==1].hist(ax=axis, bins=20)

        axis.set_title(column)
    fig.tight_layout()
    plt.show()

NameError: name 'epigenomes' is not defined

### PCA

In [4]:
def pca(x: np.ndarray, n_components: int = 2) -> np.ndarray:
    return PCA(n_components=n_components, random_state=42).fit_transform(x)

NameError: name 'np' is not defined

In [ ]:
tasks = {
    "x": [
        *[
            val.values
            for val in epigenomes.values()
        ],
        *[
            val.values
            for val in sequences.values()
        ]
    ],
    "y": [
        *[
            val.values.ravel()
            for val in labels.values()
        ],
        *[
            val.values.ravel()
            for val in labels.values()
        ]
    ],
    "titles": [
        "Epigenomes promoters",
        "Epigenomes enhancers",
        "Sequences promoters",
        "Sequences enhancers"
    ]
}

xs = tasks["x"]
ys = tasks["y"]
titles = tasks["titles"]

assert len(xs) == len(ys) == len(titles)

for x, y in zip(xs, ys):
    assert x.shape[0] == y.shape[0]

In [ ]:
colors = np.array([
    "tab:blue",
    "tab:orange",
])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(32, 8))

for x, y, title, axis in tqdm(zip(xs, ys, titles, axes.flatten()), desc="Computing PCAs", total=len(xs)):
    axis.scatter(*pca(x).T, s=1, color=colors[y.astype(int)])
    axis.xaxis.set_visible(False)
    axis.yaxis.set_visible(False)
    axis.set_title(f"PCA decomposition - {title}")
plt.savefig("./img/PCA decomposition")
plt.show()